Cette évaluation porte sur le jeu de données "german_credit_data.csv". Ce dernier contient des informations sur plus de 1000 individus ayant souscrits à un prêt bancaire. La variable duration indique la durée de l'emprunt en mois.
Le principal objectif de l'examen est d'utiliser des méthodes d'exploration, de manipulation et de préparation de données dans l'optique d'entraîner un algorithme de régression pour prédire l'espérance de vie d'un pays

- (a) Importer le module pandas sous le nom pd
- (b) Lire le fichier "german_credit_data.csv" dans un DataFrame appelé df en précisant que la première colonne contient les indices.
- (c) Afficher un aperçu et une première description des variables du jeu de données.

In [2]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer = 'german_credit_data.csv',   # chemin du fichier
                           sep = ',',                    # caractère séparant les valeurs
                           header = 0,                   # numéro de la ligne contenant le nom des colonnes
                           index_col = 0) # nom de la colonne qui indexe les entrées 

df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car


Les variables purpose et credit_amount renseigne respectivement le motif du prêt contracté par un individu et le montant du crédit.

- (d) Quel est le montant moyen du crédit suivant les différents motifs invoqués ?

In [386]:
import numpy as np

credit_purpose = df[['Purpose', 'Credit amount']].groupby("Purpose").mean()
credit_purpose
print(credit_purpose)

# Si besoin d'arrondir
credit_purpose_arrondi=credit_purpose
credit_purpose_arrondi ['Credit amount']=credit_purpose_arrondi ['Credit amount'].astype('int')
print(credit_purpose_arrondi)

                     Credit amount
Purpose                           
business               4158.041237
car                    3768.192878
domestic appliances    1498.000000
education              2879.203390
furniture/equipment    3066.988950
radio/TV               2487.653571
repairs                2728.090909
vacation/others        8209.333333
                     Credit amount
Purpose                           
business                      4158
car                           3768
domestic appliances           1498
education                     2879
furniture/equipment           3066
radio/TV                      2487
repairs                       2728
vacation/others               8209


* (e) Quel est le motif revenant le plus souvent ? 

In [8]:
# Calculer les occurences de chaque motif
df['Purpose'].value_counts(normalize=True)

car                    0.337
radio/TV               0.280
furniture/equipment    0.181
business               0.097
education              0.059
repairs                0.022
vacation/others        0.012
domestic appliances    0.012
Name: Purpose, dtype: float64

In [ ]:
# Autre méthode : afficher le motif le plus récurent & son nombre d'occurence
df[['Purpose']].describe()

df.isna().mean()*100 # renseigne le % de manquant

* (f) Afficher les informations concernant l'individu le plus âgé. Quelle est la durée de son prêt ?
* (g) Qu'en est-il de l'individu du plus jeune ?

In [13]:
# Info sur le plus vieux
df.loc[df['Age'].idxmax]

Age                       24
Sex                     male
Job                        2
Housing                  own
Saving accounts     moderate
Checking account    moderate
Credit amount           5595
Duration                  72
Purpose             radio/TV
Name: 677, dtype: object

In [449]:
# Vérifie qu'il n'y a pas d'age manquant 
df[df[['Age']].isna().any(axis = 1)]

# On remplace par la valeur moyenne juste à titre d'exemple pour ne pas fausser la suite de l'exercice
df['Age'] = df['Age'].fillna(df['Age'].mean())

df.loc[df['Age']==max(df['Age'])][['Duration']]


,Duration
330,24
536,6


- (h) La variable duration renseigne la durée des emprunts. Transformez-la en variable numérique en supprimant -month pour chaque individu.
   Pour cette question, appliquer à chaque observation de la colonne duration une fonction qui supprime les 6 derniers éléments

In [12]:
# Je garde tous les caractères sauf les 6 derniers caractère
df['duration'].apply(lambda x : x[:-6]).astype(int)

# autre manipulation
def delete_string(x):
    return (x[:-6])

df['duration']=df['duration'].apply(delete_string)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment
996,40,male,3,own,little,little,3857,30,car
997,38,male,2,own,little,NaN,804,12,radio/TV
998,23,male,2,free,little,little,1845,45,radio/TV


- (i) Créer une nouvelle variable nommée duration_categ contenant 3 modalités : court-terme pour tous les prêts d'une durée inférieure ou égale à 10 mois, moyen-terme pour tous les prêts d'une durée strictement supérieure à 10 mois et inférieure ou égale à 30 mois, long-terme pour tous les prêts d'une durée strictement supérieure à 30 mois.

In [23]:
# Créer une fonction qui attribuera court/moyen/long-terme comme valeur selon la durée contenue dans la colonne Duration
def categ(row):
    if row['Duration']<=10:
        return 'court-terme'
    if 30>row['Duration']>10:
        return 'moyen-terme'
    if row['Duration']>30:
        return 'long-terme'
    
df['duration_categ']=df.apply(categ, axis=1)
print(df)

# Alternative
## df['duration_categ']=pd.cut(df['duration'], blns=[0,10,30,72], labels=['court-terme','moyen-terme','long-terme'])

# Alternative avec np.select
## df['duration_categ']=np.select([df.duration<=10, df.duration>30],['court terme', 'long terme'],default='moyen terme')

     Age     Sex  Job Housing Saving accounts Checking account  Credit amount  \
0     67    male    2     own             NaN           little           1169   
1     22  female    2     own          little         moderate           5951   
2     49    male    1     own          little              NaN           2096   
3     45    male    2    free          little           little           7882   
4     53    male    2    free          little           little           4870   
..   ...     ...  ...     ...             ...              ...            ...   
995   31  female    1     own          little              NaN           1736   
996   40    male    3     own          little           little           3857   
997   38    male    2     own          little              NaN            804   
998   23    male    2    free          little           little           1845   
999   27    male    2     own        moderate         moderate           4576   

     Duration              

La variable housing renseigne 3 catégories : own (propriétaire), rent (locataire) et free (libre).

- (j) En majorité, les propriétaires ont-ils des prêts moyen-terme ?

In [21]:
pd.crosstab(df['Housing'],df['duration_categ'], normalize=0)

# On fait apparaître le nombre d'occurence des prêts court/moyen/long-terme des individus qui sont propriétaires 
emprunt_proprietaire=df[(df['Housing'] == 'own')]['duration_categ'].value_counts()
print(emprunt_proprietaire)

print ("\n", 'Les propriétaires ont principalement des emprunts à moyen-terme. ')

duration_categ,court-terme,long-terme,moyen-terme
Housing,,,
free,0.130841,0.373832,0.495327
own,0.182085,0.164464,0.653451
rent,0.191860,0.122093,0.686047


Preprocessing et cleaning des données
La phase de préparation des données regroupe les activités liées à la construction de l'ensemble précis des données à analyser, faite à partir des données brutes. Elle inclut ainsi le classement des données en fonction de critères choisis, le nettoyage des données, et surtout leur recodage pour les rendre compatibles avec les algorithmes qui seront utilisés.

- (k) Pour la variablechecking_account, remplacer les modalités 'little', 'moderate' et 'rich' par 0,1 et 2.

In [397]:
# Remplacer les modalités 'little', 'moderate' et 'rich' par 0,1 et 2 dans la colonne 'checking_acount'
df['checking_accoun']=df['checking_accoun'].replace(['little','moderate','rich'], ['0','1','2'])
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,0,1169,6,radio/TV
1,22,female,2,own,little,1,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,0,7882,42,furniture/equipment
4,53,male,2,free,little,0,4870,24,car
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment
996,40,male,3,own,little,0,3857,30,car
997,38,male,2,own,little,NaN,804,12,radio/TV
998,23,male,2,free,little,0,1845,45,radio/TV


* (l) Pour la variable`saving_accounts`, remplacer les modalités 'little', 'moderate', 'quite rich' et rich' par 0,1,2 et 3. 

In [398]:
# Remplacer les modalités 'little', 'moderate' et 'rich' par 0,1 et 2 dans la colonne 'saving_acount'

df['saving_accounts']=df['saving_accounts'].replace(['little','moderate', 'quite rich','rich'], ['0','1','2','3'])
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,0,1169,6,radio/TV
1,22,female,2,own,0,1,5951,48,radio/TV
2,49,male,1,own,0,NaN,2096,12,education
3,45,male,2,free,0,0,7882,42,furniture/equipment
4,53,male,2,free,0,0,4870,24,car
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,0,NaN,1736,12,furniture/equipment
996,40,male,3,own,0,0,3857,30,car
997,38,male,2,own,0,NaN,804,12,radio/TV
998,23,male,2,free,0,0,1845,45,radio/TV


- (m) Remarquez-vous la présence de doublons dans les données ?

In [392]:
print(' Il y a : ', df.duplicated().sum(), ' doublons dans les données')


 Il y a :  0  doublons dans les données


- (n) Afficher le nombre de valeurs manquantes pour chaque colonne de df.

In [393]:
print(df.isna().sum(axis = 0))

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Duration2             0
duration_categ       40
dtype: int64


Certaines variables comme 'housing' ou 'purpose' sont des variables catégorielles. En effet, elles contiennent un nombre fini de modalités (3 pour la première par exemple).

Dans le cas de ces variables, il est possible de remplacer les valeurs manquantes dans ces colonnes par l'élément le plus fréquent.
En statistique, cette valeur s'appelle le mode, et la méthode du même nom permet de retourner cet/ces eléments à l'intérieur d'une pandas Series.

- (o) Dans df, remplacer les valeurs manquantes de la variable housing par le mode.

In [438]:
# On remplace les valeurs manquantes (NaN) de la colonne 'housing' par le mode de la colonne 'housing'
df['Housing'] = df['Housing'].fillna(df['Housing'].mode()[0])
print(df)

x=df['Housing'].mode()
x

     Age     Sex  Job Housing Saving accounts Checking account  Credit amount  \
0     67    male    2     own               0                0           1169   
1     22  female    2     own               0                1           5951   
2     49    male    1     own               0              NaN           2096   
3     45    male    2    free               0                0           7882   
4     53    male    2    free               0                0           4870   
..   ...     ...  ...     ...             ...              ...            ...   
995   31  female    1     own               0              NaN           1736   
996   40    male    3     own               0                0           3857   
997   38    male    2     own               0              NaN            804   
998   23    male    2    free               0                0           1845   
999   27    male    2     own               1                1           4576   

     Duration              

0    own
dtype: object

* (p) Dans **`df`**, remplacer les valeurs manquantes de la variable `age` par la moyenne.

In [404]:
df['age'] = df['age'].fillna(df['age'].mean())
print(df)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,0,0,1169,6,radio/TV
1,22,female,2,own,0,1,5951,48,radio/TV
2,49,male,1,own,0,NaN,2096,12,education
3,45,male,2,free,0,0,7882,42,furniture/equipment
4,53,male,2,free,0,0,4870,24,car
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,0,NaN,1736,12,furniture/equipment
996,40,male,3,own,0,0,3857,30,car
997,38,male,2,own,0,NaN,804,12,radio/TV
998,23,male,2,free,0,0,1845,45,radio/TV


In [402]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer = 'german_credit_data.csv',   # chemin du fichier
                           sep = ',',                    # caractère séparant les valeurs
                           header = 0,                   # numéro de la ligne contenant le nom des colonnes
                           index_col = 0) # nom de la colonne qui indexe les entrées 

df.head()

df['Checking account']=df['Checking account'].replace(['little','moderate','rich'], ['0','1','2'])
df

df['Saving accounts']=df['Saving accounts'].replace(['little','moderate', 'quite rich','rich'], ['0','1','2','3'])
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,67,male,2,own,NaN,0,1169,6,radio/TV
1,22,female,2,own,0,1,5951,48,radio/TV
2,49,male,1,own,0,NaN,2096,12,education
3,45,male,2,free,0,0,7882,42,furniture/equipment
4,53,male,2,free,0,0,4870,24,car
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,0,NaN,1736,12,furniture/equipment
996,40,male,3,own,0,0,3857,30,car
997,38,male,2,own,0,NaN,804,12,radio/TV
998,23,male,2,free,0,0,1845,45,radio/TV


Apprentissage des données
Une équipe de Data Scientists a déjà entraîné un modèle de régression prédictif sur des données en plus grand nombre et identiques aux données sur lesquelles vous avez travaillé. Après envoi de vos données sans la variable duration ils ont essayé de prédire la durée des prêts de vos 1000 individus.

Les résultats des prédictions sont disponibles dans le fichier 'predictions_german.csv', qui contient également l'ID des clients.

- (q) Lire le fichier "predictions_german.csv" dans un DataFrame appelé predictions en précisant que la première colonne contient les indices.
- (r) En fusionnant df et predictions, créer un dataframe df_pred contenant les informations dont nous disposons ainsi que les prédictions sur les 1000 individus.

In [ ]:
predictions = pd.read_csv(filepath_or_buffer = 'predictions_german.csv',   # chemin du fichier
                           sep = ',',                    # caractère séparant les valeurs
                           header = 0,                   # numéro de la ligne contenant le nom des colonnes
                           index_col = 0) # nom de la colonne qui indexe les entrées 

# On vérifie sur l'import a été correctement réalisé
predictions.head()

In [ ]:
# join
df_pred = df.join(predictions)

# Merging
df_pred=df.merge(right = predictions, on = '??? nom colonne à renseigner', how = 'left')

# 2nde alternative
df_pred = pd.concat([df,predictions], axis = 1)

Évaluation du modèle de régression 
- (s) Créer une nouvelle variable error renseignant la différence entre les variables duration et predictions.

In [407]:
df_pred['error'] = df['duration'] - df['predictions']

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,NEW
0,67,male,2,own,0,0,1169,6,radio/TV,65
1,22,female,2,own,0,1,5951,48,radio/TV,20
2,49,male,1,own,0,NaN,2096,12,education,48
3,45,male,2,free,0,0,7882,42,furniture/equipment,43
4,53,male,2,free,0,0,4870,24,car,51
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,0,NaN,1736,12,furniture/equipment,30
996,40,male,3,own,0,0,3857,30,car,37
997,38,male,2,own,0,NaN,804,12,radio/TV,36
998,23,male,2,free,0,0,1845,45,radio/TV,21


- (t) Combien d'individus ont vu leur durée de prêt sous-estimée par le modèle ?

In [431]:
('Il y a : ', len(df_pred[df_pred['error'] < 0], ' personnes qui ont vu leur durée de prêt sous-estimée par le modèle')

918